In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('../001/train.pk1')

In [3]:
train_pkl.shape

(850, 11)

In [4]:
# 年齢、性別は削除
train_pkl.drop(['Age', 'Gender_Male'], axis=1, inplace=True)

In [5]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0


In [6]:
train_pk2 = train_pkl.copy()
train_pkl.shape

(850, 9)

In [14]:
train_pkl = train_pk2.copy()
loop = train_pkl.shape[1] - 1
for i in range(loop):
    for j in range(i+1, loop):
        colname = train_pkl.columns[i] + "_" + train_pkl.columns[j] 
        train_pkl["sub" + "_" + colname] = train_pkl.iloc[:, i] - train_pkl.iloc[:, j]

In [15]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,sub_T_Bil_D_Bil,...,sub_ALT_GPT_AST_GOT,sub_ALT_GPT_TP,sub_ALT_GPT_Alb,sub_ALT_GPT_AG_ratio,sub_AST_GOT_TP,sub_AST_GOT_Alb,sub_AST_GOT_AG_ratio,sub_TP_Alb,sub_TP_AG_ratio,sub_Alb_AG_ratio
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0,0.636719,...,-8.261629,6.651211,10.356289,12.460781,14.912840,18.617918,20.722410,3.705078,5.808594,2.105469
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,1,0.808105,...,-13.718857,44.142838,47.982681,50.281998,57.861694,61.701538,64.000854,3.839844,6.140625,2.298828
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0,0.576172,...,-17.988169,6.675793,10.136730,11.850109,24.663961,28.124899,29.838278,3.460938,5.175781,1.712891
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,1,0.692383,...,-20.162487,27.378849,30.734318,33.359318,47.541336,50.896805,53.521805,3.355469,5.980469,2.625000
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,0,1.537109,...,-149.437286,14.736954,17.504532,19.546524,164.174240,166.941818,168.983810,2.767578,4.808594,2.042969


In [16]:
train_pkl.describe()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,disease,sub_T_Bil_D_Bil,...,sub_ALT_GPT_AST_GOT,sub_ALT_GPT_TP,sub_ALT_GPT_Alb,sub_ALT_GPT_AG_ratio,sub_AST_GOT_TP,sub_AST_GOT_Alb,sub_AST_GOT_AG_ratio,sub_TP_Alb,sub_TP_AG_ratio,sub_Alb_AG_ratio
count,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,...,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000,850.000000
mean,1.607422,0.606445,272.211334,41.978474,61.018120,7.031250,3.517578,1.144531,0.444706,1.000977,...,-19.039644,34.946140,38.461033,40.833908,53.985783,57.500675,59.873547,3.515625,5.886719,2.373047
std,2.443359,1.568359,200.348511,145.927368,110.125389,0.840820,0.560059,0.227295,0.497226,1.630859,...,111.146957,146.014297,145.977509,145.926224,110.188789,110.175346,110.130692,0.579102,0.809082,0.458984
min,0.585938,0.034851,163.261841,3.924518,11.283497,4.859375,2.181641,0.626953,0.000000,-11.265625,...,-764.726074,-3.098919,0.438190,2.629596,4.377247,7.711231,10.256153,1.929688,3.812500,1.188477
25%,0.781372,0.144928,213.991409,13.644659,21.239157,6.730469,3.138672,1.004883,0.000000,0.599609,...,-32.133928,6.649300,10.161786,12.480119,14.015506,17.549161,19.951804,3.138184,5.407227,2.093750
50%,0.835693,0.194092,220.092506,16.638000,27.056026,6.914062,3.621094,1.205078,0.000000,0.663086,...,-9.450084,9.922703,13.349444,15.657165,20.427270,23.737206,25.827234,3.515625,5.886719,2.347656
75%,1.197021,0.335388,229.455933,23.056081,56.461569,7.535156,3.712891,1.288086,1.000000,0.826294,...,-5.243752,16.189765,19.644595,21.771520,49.551988,53.065876,55.280540,3.890625,6.269531,2.648926
max,23.015625,17.687500,2108.483643,1423.186523,814.439392,8.742188,5.015625,1.821289,1.000000,11.023438,...,1069.522827,1417.139648,1420.030273,1421.993164,807.662048,811.255798,813.179626,5.550781,7.718750,3.748047


In [19]:
disease_bk = train_pkl.disease.values

In [20]:
train_pkl.drop('disease', axis=1, inplace=True)

In [21]:
train_pkl['disease'] = disease_bk

In [22]:
train_pkl.head()

,T_Bil,D_Bil,ALP,ALT_GPT,AST_GOT,TP,Alb,AG_ratio,sub_T_Bil_D_Bil,sub_T_Bil_ALP,...,sub_ALT_GPT_TP,sub_ALT_GPT_Alb,sub_ALT_GPT_AG_ratio,sub_AST_GOT_TP,sub_AST_GOT_Alb,sub_AST_GOT_AG_ratio,sub_TP_Alb,sub_TP_AG_ratio,sub_Alb_AG_ratio,disease
0,0.787109,0.150513,220.178696,13.467617,21.729246,6.816406,3.111328,1.006836,0.636719,-219.391586,...,6.651211,10.356289,12.460781,14.912840,18.617918,20.722410,3.705078,5.808594,2.105469,0
1,1.003906,0.195679,221.218414,51.033463,64.752319,6.890625,3.050781,0.751465,0.808105,-220.214508,...,44.142838,47.982681,50.281998,57.861694,61.701538,64.000854,3.839844,6.140625,2.298828,1
2,0.657227,0.081299,320.770538,12.625011,30.613180,5.949219,2.488281,0.774902,0.576172,-320.113312,...,6.675793,10.136730,11.850109,24.663961,28.124899,29.838278,3.460938,5.175781,1.712891,0
3,0.906738,0.214233,369.278168,34.347599,54.510086,6.968750,3.613281,0.988281,0.692383,-368.371429,...,27.378849,30.734318,33.359318,47.541336,50.896805,53.521805,3.355469,5.980469,2.625000,1
4,1.735352,0.197754,222.782028,20.572891,170.010178,5.835938,3.068359,1.026367,1.537109,-221.046677,...,14.736954,17.504532,19.546524,164.174240,166.941818,168.983810,2.767578,4.808594,2.042969,0


## 訓練データとテストデータに分割

In [23]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [24]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1]
train_y = train.disease.values

In [27]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

## 訓練データで訓練

In [52]:
XGB = XGBClassifier(lambda_l1=0, lamba_l2=15, max_depth=7)

In [53]:
XGB = XGB.fit(train_X, train_y, early_stopping_rounds=20, eval_metric='auc', eval_set=[(test_x, test_y)])

[0]	validation_0-auc:0.907066
Will train until validation_0-auc hasn't improved in 20 rounds.
[1]	validation_0-auc:0.9292
[2]	validation_0-auc:0.947633
[3]	validation_0-auc:0.947563
[4]	validation_0-auc:0.953149
[5]	validation_0-auc:0.957478
[6]	validation_0-auc:0.959293
[7]	validation_0-auc:0.962226
[8]	validation_0-auc:0.965717
[9]	validation_0-auc:0.9646
[10]	validation_0-auc:0.963413
[11]	validation_0-auc:0.963692
[12]	validation_0-auc:0.962296
[13]	validation_0-auc:0.96467
[14]	validation_0-auc:0.964251
[15]	validation_0-auc:0.963553
[16]	validation_0-auc:0.961598
[17]	validation_0-auc:0.962016
[18]	validation_0-auc:0.962994
[19]	validation_0-auc:0.963971
[20]	validation_0-auc:0.963692
[21]	validation_0-auc:0.962994
[22]	validation_0-auc:0.963553
[23]	validation_0-auc:0.965508
[24]	validation_0-auc:0.964809
[25]	validation_0-auc:0.96467
[26]	validation_0-auc:0.965228
[27]	validation_0-auc:0.96453
[28]	validation_0-auc:0.965647
Stopping. Best iteration:
[8]	validation_0-auc:0.96571

In [54]:
XGB.feature_importances_

array([0.04254039, 0.01664801, 0.02561088, 0.01098312, 0.02410373,
       0.01250019, 0.00787819, 0.01266339, 0.01683821, 0.01633656,
       0.01971731, 0.01973436, 0.01243057, 0.01357868, 0.15071903,
       0.00949494, 0.02093574, 0.01870111, 0.01509844, 0.01416896,
       0.01193526, 0.01094405, 0.01137072, 0.0126647 , 0.02943373,
       0.01091315, 0.02136728, 0.09002731, 0.04085169, 0.08956975,
       0.0169444 , 0.03902404, 0.10072727, 0.01108229, 0.01465674,
       0.00780585], dtype=float32)

In [55]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train_X.iloc[:, :].columns),
    reverse=True)

[(0.151, 'sub_T_Bil_AG_ratio'),
 (0.101, 'sub_AST_GOT_AG_ratio'),
 (0.09, 'sub_ALT_GPT_TP'),
 (0.09, 'sub_ALT_GPT_AG_ratio'),
 (0.043, 'T_Bil'),
 (0.041, 'sub_ALT_GPT_Alb'),
 (0.039, 'sub_AST_GOT_Alb'),
 (0.029, 'sub_ALP_Alb'),
 (0.026, 'ALP'),
 (0.024, 'AST_GOT'),
 (0.021, 'sub_D_Bil_ALT_GPT'),
 (0.021, 'sub_ALT_GPT_AST_GOT'),
 (0.02, 'sub_T_Bil_AST_GOT'),
 (0.02, 'sub_T_Bil_ALT_GPT'),
 (0.019, 'sub_D_Bil_AST_GOT'),
 (0.017, 'sub_T_Bil_D_Bil'),
 (0.017, 'sub_AST_GOT_TP'),
 (0.017, 'D_Bil'),
 (0.016, 'sub_T_Bil_ALP'),
 (0.015, 'sub_TP_AG_ratio'),
 (0.015, 'sub_D_Bil_TP'),
 (0.014, 'sub_T_Bil_Alb'),
 (0.014, 'sub_D_Bil_Alb'),
 (0.013, 'sub_ALP_TP'),
 (0.013, 'TP'),
 (0.013, 'AG_ratio'),
 (0.012, 'sub_T_Bil_TP'),
 (0.012, 'sub_D_Bil_AG_ratio'),
 (0.011, 'sub_TP_Alb'),
 (0.011, 'sub_ALP_AST_GOT'),
 (0.011, 'sub_ALP_ALT_GPT'),
 (0.011, 'sub_ALP_AG_ratio'),
 (0.011, 'ALT_GPT'),
 (0.009, 'sub_D_Bil_ALP'),
 (0.008, 'sub_Alb_AG_ratio'),
 (0.008, 'Alb')]

## テストデータで実行

In [56]:
test_x.shape, test_y.shape

((170, 36), (170,))

In [57]:
pred_y = XGB.predict(test_x)

In [58]:
confusion_matrix(test_y, pred_y)

array([[87,  6],
       [15, 62]], dtype=int64)

In [59]:
accuracy_score(test_y, pred_y)

0.8764705882352941